# Implement Low-Rank Adaptation for LLMs in PyTorch

https://lightning.ai/lightning-ai/studios/code-lora-from-scratch?tab=overview&layout=column&path=cloudspaces%2F01hm9hypqc6y1hrapb5prmtz0h&y=3&x=0

# 1 - Understanding LoRA

Pretrained LLMs are often dubbed foundation models because of their versatility across various tasks. However, it is often useful to adapt a pretrained LLM for a specific dataset or task, which we can accomplish via finetuning.

Finetuning allows the model to adapt to specific domains without costly pretraining, yet updating all layers can still be computationally expensvie, espcially for larger models.

LoRA offers a more efficient alternative to regular finetuning. As discussed in more detail in the paper [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685), LoRA approximates a layer's weight changes during training, $\Delta W$, in a low-rank format.

for instance, whereas in regular finetuning, we compute the weight updates of a weight matrix $W$ as $\Delta W$, in LoRA, we approximate $\Delta W$ through the matrix multiplication of two smaller matrices $A B$, as illustrated in the figure below. (If you are familiar with PCA or SVD, consider this as decomposing $\Delta W$ in $A$ and $B$)

<table>
    <tr>
        <td><img src="./images_3/lora_summary.png" width="800"/></td>
    </tr>
</table>

Note that $r$, in the figure above, is a hyperparameter here that we can use to specify the rank of the low-rank matrices used for adaptation. A smaller $r$ leads to a simpler low-rank matrix, which results in fewer parameters to learn during adaptation. This can lead to faster training and potentially reduced computational requirements. However, with a smaller $r$, the capacity of the low-rank matrix to capture task specific information decreases.

To provide a concrete example, suppose the weight matrix $W$ of a given layer has a size $5,000 \times 10,000$ (50M parameters in total). If we choose a rank $r=8$, we initialize two matrices: the $5,000 \times 8$-dimensional matrix $B$ and the $8 \times 10,000$-dimensional matrix $A$. Added together, $A$ and $B$ have only $40,000 + 80,000 = 120,000$ parameters, which is 400 times smaller than the 50M parameters in regular finetuning via $\Delta W$.

In practice, it's important to experiment with different $r$ values to find the right balance to achieve the desired performance on the new task.

# 2 - Coding LoRA from scratch

Since conceptual explanations can sometimes be abstract, let's now implement LoRA ourselves to get a btter idea of how it works. In code, we can implement a LoRA layer as follows:

In [1]:
import torch


class LoRALayer(torch.nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        self.W_a = torch.nn.Parameter(torch.randn(in_dim, rank) * std_dev)
        self.W_b = torch.nn.Parameter(torch.zeros(rank, out_dim))
        self.alpha = alpha

    def forward(self, x):
        x = self.alpha * (x @ self.W_a @ self.W_b)
        return x

In the code above, `in_dim` is the input dimension of the layer we want to modify using LoRA, and `out_dim` is the respective output dimension of that layer.

We previously discussed that the rank of the matrices $A$ and $B$ is a hyperparamter that controls the complexity and the number of additional parameters introduced by LoRA.

However, looking at the code above, we added another hyperparameter, the scaling factor `alpha`. This factor determines the magnitude of the changes introduced by the LoRA layer to the model's existing weights: `alpha * (x @ A @ B)`. A higher value of `alpha` means larger adjustments to the model's behaviour, while a lower value results in more subtle changes.

Another thing to note is that we initialized `A` with small values from a random distribution. Here, the standard deviation of this distribution is determined by the square root of the rank (this choice ensures that the initial values in `A` are not too large). However, we initialized `B` with zeros. The rationale here is that at the beginning of the training, before `A` and `B` are updated via backpropagation, the `LoRALayer` does not impact the original weights because `AB=0` if `B=0`

Note that LoRA is usually applied to a neural network's linear (feedforward) layers. For example, suppose we have a simple PyTorch model or module with two linear layers (e.g., this could be the feedforward module of a transformer block). And suppose this module's forward method looks like as follows:

```python

def forward(self, x):
    x = self.linear_1(x)
    x = F.relu(x)
    x = self.linear_2(x)
    return x

```

If we use LoRA, we'd add the LoRA updates to these linear layer outputs as follows:

```python

def forward(self, x):
    x = self.linear_1(x) + self.lora_1(x)
    x = F.relu(x)
    x = self.linear_2(x) + self.lora_2(x)
    return logits

```

In code, when implementing LoRA by modifying existing PyTorch models, an easy way to implement such a LoRA-modification of the linear layers is to replace each Linear layer with a `LinearWithLoRA` layer that combines the `Linear` layer with our previous `LoRALayer` implementation:

In [2]:
class LinearWithLoRA(torch.nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        return self.linear(x) + self.lora(x)

## 2.1 - Example

In [4]:
torch.manual_seed(123)

# a simple linear layer with 10 inputs and 1 output
# requires_grad=False makes it non-trainable
linear_layer = torch.nn.Linear(10, 1)
linear_layer.weight.requires_grad = False
linear_layer.bias.requires_grad = False

# a simple example input
x = torch.rand((1, 10))

linear_layer(x)

tensor([[-0.5745]])

Replace linear layer with LoRA layer

In [5]:
lora_layer = LinearWithLoRA(linear=linear_layer, rank=8, alpha=1)
lora_layer(x)

tensor([[-0.5745]], grad_fn=<AddBackward0>)

Note that the LoRA layer will not change the linear output untils it is trained because its `W_b` weight matrix is initialized to all zeros. For LoRA to take effect, we have to train the model. Below is a simple toy example.

Let's simulate a simple weight update:

In [6]:
lora_layer.lora.W_b = torch.nn.Parameter(lora_layer.lora.W_b + 0.01 * x[0])


We can now see that the output has changed:

In [7]:
lora_layer(x)

tensor([[-0.5863, -0.5758, -0.5779, -0.5800, -0.5814, -0.5766, -0.5887, -0.5811,
         -0.5859, -0.5892]], grad_fn=<AddBackward0>)

## 2.2 - Summary

These concepts above are summarized in the following figure:

<table>
    <tr>
        <td><img src="./images_3/lora_from_scratch_summary.jpg" width="1000"/></td>
    </tr>
</table>

In practice, to equip and finetune a model with LoRA, all we have to do is replace its pretrained `Linear` layers with our new `LinearWithLoRA` layer. 

# 3 - Finetuning with LoRA -- A hands-on example

LoRA is a method that can be applied to various types of neural networks, not just generative models like GPT or image generation models. For this hands-on example, we will train a small BERT model for text classification because classification accuracy is easier to evaluate than generated text.

In particular, we will use a pretrained DistilBERT model from the transformers library:

```python

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2)
    
```

Since we only want to train the new LoRA weights, we freeze all model parameters by setting `requires_grad` to `False` for all trainable parameters:

```python

for param in model.parameters():
    param.requires_grad = False

```

## 3.1 - Loading the dataset into DataFrames

In [10]:
import os
from datasets import load_dataset

import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

import pandas as pd
import torch

For the data, we are going to use a sampled version of the IMDB dataset that we have prepared by running the `notes-ai/Foundation models/Fine-tuning/Parameter-efficient Finetuning/local_dataset_utilities.py` script.

Since this notebook is also a modified version of another Lightning blog post that uses the same data, I have prepared a sampled version of the dataset to test it

In [ ]:
df_train = pd.read_csv("train_sample.csv")
df_val = pd.read_csv("val_sample.csv")
df_test = pd.read_csv("test_sample.csv")